In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from IPython.display import display
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import time
import glob
import sys
import os
import gc

In [4]:
fold_n=5
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=10)
%precision 4
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

In [6]:
train = pd.read_csv(r'D:\PycharmProjects\EDA-ML\santander\train.csv')
test =  pd.read_csv(r'D:\PycharmProjects\EDA-ML\santander\test.csv')

In [7]:
train.sample()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
18758,37583,2,45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91046.970000000001164,0


In [24]:
targ=pd.DataFrame([train['TARGET'].value_counts(),(train['TARGET'].value_counts()/train.shape[0])*100]).T
targ.columns = ['Target', 'Target%']

In [25]:
targ

,Target,Target%
0,73012.0,96.04314654038410
1,3008.0,3.95685345961589


In [27]:
train.shape

(76020, 371)

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [29]:
train.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,...,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,76020.000000000000000,7.602000000000000e+04,76020.000000000000000
mean,75964.050723493812256,-1523.199276506182514,33.212865035516970,86.208265193370153,72.363067087608528,119.529632202052071,3.559129834254144,6.472697711128651,0.412946329913181,0.567352407261247,...,7.935823993685871,1.365146014206788,12.215579715864248,8.784074191002366,31.505324388318854,1.858574980268350,76.026165351223355,56.614351223362270,1.172358094303766e+05,0.039568534596159
std,43781.947379493780318,39033.462363863283827,12.956485816411226,1614.757312723822224,339.315830906625990,546.266294057292043,93.155748705970623,153.737065554838694,30.604863766118754,36.513513214598269,...,455.887217988078078,113.959637254642558,783.207398896920608,538.439210544997650,2013.125392638054336,147.786584178518410,4040.337841907307848,2852.579397358309507,1.826645985028727e+05,0.194944519232523
min,1.000000000000000,-999999.000000000000000,5.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,...,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,5.163750000000000e+03,0.000000000000000
25%,38104.750000000000000,2.000000000000000,23.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,...,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,6.787061250000000e+04,0.000000000000000
50%,76043.000000000000000,2.000000000000000,28.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,...,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,1.064091600000000e+05,0.000000000000000
75%,113748.750000000000000,2.000000000000000,40.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,...,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,1.187562525000000e+05,0.000000000000000
max,151838.000000000000000,238.000000000000000,105.000000000000000,210000.000000000000000,12888.030000000000655,21024.810000000001310,8237.819999999999709,11073.569999999999709,6600.000000000000000,6600.000000000000000,...,50003.879999999997381,20385.720000000001164,138831.630000000004657,91778.729999999995925,438329.219999999972060,24650.009999999998399,681462.900000000023283,397884.299999999988358,2.203473876000000e+07,1.000000000000000
